# Predictinng Request For Change(RFC)

## Predict RFC (Request for change) and possible failure / misconfiguration of ITSM assets.

### DB connection

In [50]:
from sqlalchemy import create_engine
import pandas as pd

db_host = '18.136.56.185'
username ='dm_team'
pwd='dm_team123#'
db_name='project_itsm'

#Connecting to SQL DB
conn =create_engine('mysql+pymysql://'+username+':'+pwd+'@'+db_host+'/'+db_name)
conn.table_names()

['dataset_list']

### Loading data

In [76]:
query ='select * from dataset_list '
df =pd.read_sql(query,conn)
print(df.shape)
df.head(2)
df.columns

(46606, 25)


Index(['CI_Name', 'CI_Cat', 'CI_Subcat', 'WBS', 'Incident_ID', 'Status',
       'Impact', 'Urgency', 'Priority', 'number_cnt', 'Category', 'KB_number',
       'Alert_Status', 'No_of_Reassignments', 'Open_Time', 'Reopen_Time',
       'Resolved_Time', 'Close_Time', 'Handle_Time_hrs', 'Closure_Code',
       'No_of_Related_Interactions', 'Related_Interaction',
       'No_of_Related_Incidents', 'No_of_Related_Changes', 'Related_Change'],
      dtype='object')

In [70]:
df.head(5)
df.columns
print(Counter(df['No_of_Related_Changes']))
df.dtypes

Counter({'1': 46582, '2': 21, '3': 2, '9': 1})


CI_Name                       object
CI_Cat                        object
CI_Subcat                      int64
WBS                            int64
Incident_ID                   object
Status                        object
Impact                        object
Urgency                       object
Priority                      object
number_cnt                    object
Category                       int64
KB_number                     object
Alert_Status                  object
No_of_Reassignments           object
Open_Time                     object
Reopen_Time                   object
Resolved_Time                 object
Close_Time                    object
Handle_Time_hrs               object
Closure_Code                  object
No_of_Related_Interactions    object
Related_Interaction           object
No_of_Related_Incidents       object
No_of_Related_Changes         object
Related_Change                object
dtype: object

### Data Munging

####  "CI_Subcat" column 

In [53]:
from collections import Counter
import numpy as np
from scipy import stats

mode_val=stats.mode(df['CI_Subcat']).mode

# CI_Subcat field cleaning 
Counter(df.CI_Subcat)
df.CI_Subcat.isnull().sum()

#Replacing empty srting values of 111 records of CI_Subcat column  with mode

df['CI_Subcat'] = np.where((df.CI_Cat == ''), mode_val ,df.CI_Subcat)
Counter(df['CI_Subcat'])

df.dropna(inplace=True)

df.isnull().sum()
Counter(df['CI_Subcat'])

Counter({'Web Based Application': 15311,
         'Desktop Application': 3876,
         'Server Based Application': 18922,
         'SAP': 1199,
         'Client Based Application': 492,
         'Citrix': 767,
         'Standard Application': 85,
         'Windows Server': 153,
         'Laptop': 1921,
         'Linux Server': 55,
         'Monitor': 212,
         'Automation Software': 55,
         'SAN': 549,
         'Banking Device': 916,
         'Desktop': 423,
         'Database': 211,
         'Oracle Server': 3,
         'Keyboard': 27,
         'Printer': 88,
         'Exchange': 130,
         'System Software': 272,
         'VDI': 9,
         'Encryption': 45,
         'Omgeving': 86,
         'MigratieDummy': 60,
         'Scanner': 64,
         'Controller': 146,
         'DataCenterEquipment': 277,
         'KVM Switches': 32,
         'Switch': 28,
         'Database Software': 4,
         'Network Component': 32,
         'Unix Server': 7,
         'Lines': 15,
      

#### Cleaning column 'Priority' ,'WBS' , 'No_of_Related_Interactions','No_of_Related_Incidents' ,'Category'

In [54]:
print(Counter(df.No_of_Related_Changes))
Counter(df.No_of_Related_Interactions)


Counter({'': 46046, '1': 536, '2': 21, '3': 2, '9': 1})


Counter({'1': 43058,
         '2': 2736,
         '3': 429,
         '14': 2,
         '7': 12,
         '4': 124,
         '5': 30,
         '370': 1,
         '9': 5,
         '11': 2,
         '54': 1,
         '': 114,
         '288': 1,
         '34': 2,
         '44': 2,
         '39': 2,
         '12': 6,
         '42': 3,
         '6': 16,
         '8': 6,
         '28': 2,
         '29': 2,
         '13': 4,
         '18': 2,
         '20': 5,
         '15': 3,
         '31': 1,
         '88': 1,
         '30': 1,
         '41': 1,
         '74': 1,
         '16': 2,
         '24': 1,
         '17': 2,
         '57': 1,
         '33': 2,
         '10': 3,
         '118': 1,
         '45': 2,
         '55': 1,
         '40': 1,
         '19': 3,
         '22': 3,
         '23': 2,
         '37': 1,
         '26': 2,
         '43': 1,
         '25': 1,
         '27': 1,
         '21': 1})

In [60]:
#------------Priority : for 'NA' values in Priority replcaing with respecctive Urgency vals
Counter(df.Priority)
df['Priority'] = np.where((df['Priority']== 'NA') , df['Urgency'],df['Priority'])

#-------------No_of_Related_Interactions : replace empty values with '1'
Counter(df.No_of_Related_Interactions)
df['No_of_Related_Interactions'] = np.where((df.No_of_Related_Interactions == ''),'1',df.No_of_Related_Interactions)

#------------No_of_Related_Incidents : replace empty values with '1'
Counter(df.No_of_Related_Incidents) 
df['No_of_Related_Incidents'] = np.where((df.No_of_Related_Incidents == ''),'1',df.No_of_Related_Incidents)

#----------Category
Counter(df.Category)

#----------No_of_Related_Changes
Counter(df.No_of_Related_Changes)
df['No_of_Reassignments'] = np.where((df.No_of_Reassignments == ''),'1',df.No_of_Reassignments)

#----------No_of_Related_Changes
df['No_of_Related_Changes'] = np.where((df.No_of_Related_Changes == ''),'1',df.No_of_Related_Changes)
print(Counter(df.No_of_Reassignments))
Counter(df.No_of_Related_Changes)

Counter({'1': 46582, '2': 21, '3': 2, '9': 1})


Counter({'1': 46582, '2': 21, '3': 2, '9': 1})

## Label Encoding of columns


In [61]:
from sklearn.preprocessing import LabelEncoder

li = ['Category','CI_Subcat','WBS']
encoder =LabelEncoder()

for i in li:
    df[i] = encoder.fit_transform(df[i])


## Define X and y : train_test_split

In [71]:
x1 = ['Category' ,'CI_Subcat' ,'WBS' ,'Priority','No_of_Related_Interactions','No_of_Related_Incidents']

In [72]:
X = df.loc[: , x1]
y =df.No_of_Related_Changes

In [73]:
from sklearn.model_selection import train_test_split as tts

X_train,X_test,y_train,y_test = tts(X ,y ,test_size=0.3 ,random_state=10)
y_train

37182    1
39573    1
1595     1
42983    1
32730    1
        ..
40059    1
28017    1
29199    1
40061    1
17673    1
Name: No_of_Related_Changes, Length: 32624, dtype: object

## PCA to determine column signficance

In [65]:
from sklearn.preprocessing import StandardScaler
scaler =StandardScaler()
X_std = scaler.fit_transform(X)

In [66]:
from sklearn.decomposition import PCA
pca =PCA()
X1 = pd.DataFrame(pca.fit_transform(X_std))

print(X.columns)
print(pca.explained_variance_ratio_)
print(pca.explained_variance_)

Index(['Category', 'CI_Subcat', 'WBS', 'Priority',
       'No_of_Related_Interactions', 'No_of_Related_Incidents',
       'No_of_Related_Changes'],
      dtype='object')
[0.21545527 0.1625542  0.15567206 0.14253404 0.13011628 0.12011841
 0.07354974]
[1.50821923 1.13790385 1.08972782 0.9977597  0.9108335  0.84084688
 0.51485921]


## Define model : MLPClassifier

In [74]:
from sklearn.neural_network import MLPClassifier

model = MLPClassifier(hidden_layer_sizes=(80,100),max_iter=600,random_state=10)
model.fit(X_train,y_train)
y_predict=model.predict(X_test)
accuracy_score(y_test,y_predict)

0.9993563152624804

## Define Model : RandomForestClassifier

In [75]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score , confusion_matrix ,classification_report

model_rf = RandomForestClassifier(max_depth= 2, n_estimators= 500)
model_rf.fit(X_train,y_train)
y_predict= model_rf.predict(X_test)

accuracy_score(y_test ,y_predict)

0.9993563152624804

In [69]:
print(confusion_matrix(y_test ,y_predict))
print(classification_report(y_test ,y_predict))

[[13973     0     0]
 [    8     0     0]
 [    1     0     0]]


C:\Users\Rakshu\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           1       1.00      1.00      1.00     13973
           2       0.00      0.00      0.00         8
           3       0.00      0.00      0.00         1

    accuracy                           1.00     13982
   macro avg       0.33      0.33      0.33     13982
weighted avg       1.00      1.00      1.00     13982



### HyperParameter tuning using GridSearchCV

In [87]:
from sklearn.model_selection import GridSearchCV

params={'n_estimators' : [5,10,100,500,250],
       'max_depth' : [2,4,8,10,15]}

cv =GridSearchCV(model_rf ,params ,cv=5)
cv.fit(X_train,y_train)

C:\Users\Rakshu\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=8,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=1000, n_jobs=None,
                                              oob_score=False,
                                              random_state=None, verbose=0,
                                              warm_start=False),
             iid='war

In [88]:
print("Best Estimators :  ", cv.best_estimator_)
print("Best Score : " , cv.best_score_)
print("Best Params : ",cv.best_params_)

Best Estimators :   RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=2, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=5,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)
Best Score :  0.9873406081412457
Best Params :  {'max_depth': 2, 'n_estimators': 5}
